In [ ]:
import pandas as pd
import re
pd.options.display.max_rows = 99
from time import time
import vaex

con = 'sqlite:///seek/jobs.db'

In [ ]:
jobs = pd.read_sql(f'''
    select jobs.*, details.details from jobs 
    left join details on jobs.id = details.job_id
    where jobs.time > {time() - 60 * 60 * 24 * 14}
    and sector like '%information%'
''', con='sqlite:///seek/jobs.db')
filtered = jobs[
    jobs.title.str.contains(r'(software|data) (developer|engineer)', flags=re.IGNORECASE) &
    ~jobs.title.str.contains(r'senior|staff|lead', flags=re.IGNORECASE)
].iloc[21:]
filtered.url = 'https://seek.com.au/job' + filtered.id.astype(str)
filtered.drop(columns=['details']).to_excel('applications.xlsx', index=False)
for i in range(len(filtered)):
    row = filtered.iloc[i].drop_duplicates()
    print('https://seek.com.au/job/' + str(row.id), row.title, row.company)

In [ ]:
import requests
import boto3
import json
import seek
import pandas as pd
import sqlite3

In [ ]:
with sqlite3.connect('./seek/jobs.db') as con:
    con.execute('alter table jobs add column salary text')
    con.commit()
    